In [1]:
import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')

In [2]:
# import matplotlib.pyplot as plt
# def plot_list(images, title=None):
#     fig, axs = plt.subplots(1, len(images), figsize=(18, 3))
#     for i in range(len(images)):
#         axs[i].imshow(mi.util.convert_to_bitmap(images[i]))
#         axs[i].axis('off')
#     if title is not None:
#         plt.suptitle(title)

In [3]:
# # Rendering resolution
# render_res = 256


# # Number of sensors
# sensor_count = 7

In [4]:
# sensors = []

# for i in range(sensor_count):
#     angle = 360.0 / sensor_count * i
#     sensors.append(mi.load_dict({
#         'type': 'perspective',
#         'fov': 45,
#         'to_world': mi.ScalarTransform4f.translate([0.5, 0.5, 0.5]) \
#                                         .rotate([0, 1, 0], angle)   \
#                                         .look_at(target=[0, 0, 0],
#                                                  origin=[0, 0, 1.3],
#                                                  up=[0, 1, 0]),
#         'film': {
#             'type': 'hdrfilm',
#             'width': render_res,
#             'height': render_res,
#             'filter': {'type': 'box'},
#             'pixel_format': 'rgba'
#         }
#     }))

In [5]:
# scene_ref = mi.load_file('./scenes/lego/scene.xml')
# ref_images = [mi.render(scene_ref, sensor=sensors[i], spp=64) for i in range(sensor_count)]
# plot_list(ref_images, 'Reference images')

In [6]:
from my_camera import OrthogonalCamera

In [7]:
image_res = [256, 256]
wavefront_size = image_res[0] * image_res[1]
camera= OrthogonalCamera(cam_origin = mi.Vector3f(0, -1, 0),cam_dir=dr.normalize(mi.Vector3f(0, 1, 0)), image_res= image_res)

In [8]:
rays = camera.spawn_image_rays()

In [9]:
from my_integrator import *
print("registering integrator...")
mi.register_integrator("nerf", lambda props: RadianceFieldPRB(props))
mi.register_integrator("mask_nerf", lambda props: MaskNeRFPRB(props))

registering integrator...


In [10]:
scene = mi.load_dict({
    'type': 'scene',
    'integrator': {
        'type': 'nerf'
    },
    'emitter': {
        'type': 'constant'
    }
    })
integrator = scene.integrator()

In [11]:
sampler = mi.load_dict({'type' : 'independent',
                        'sample_count': 1,})
sampler.seed(0, wavefront_size=wavefront_size)

In [12]:
sampler

IndependentSampler[
  base_seed = 0
  sample_count = 1
  samples_per_wavefront = 1
  wavefront_size = 65536
]

In [13]:
## 65535 numpy array
import numpy as np
active = np.ones(wavefront_size, dtype=int)

In [14]:
active

array([1, 1, 1, ..., 1, 1, 1])

In [15]:
active = mi.Bool(active)

In [16]:
active

[True, True, True, True, True, .. 65526 skipped .., True, True, True, True, True]

In [20]:
params = mi.traverse(integrator)
opt = mi.ad.Adam(lr=LR, params={'sigmat': params['sigmat'], 'sh_coeffs': params['sh_coeffs']})
params.update(opt)

for i in range(10):
    img,_,_ = integrator.sample(mode=dr.ADMode.Primal,scene=scene, ray=rays, sampler=sampler, δL=None, state_in=None, active=active)

hit [False, False, False, False, False, .. 65526 skipped .., False, False, False, False, False]
active [True, True, True, True, True, .. 65526 skipped .., True, True, True, True, True]


Exception: loop_process_state(): one of the supplied loop state variables of type Float is attached to the AD graph (i.e., grad_enabled(..) is true). However, propagating derivatives through multiple iterations of a recorded loop is not supported (and never will be). Please see the documentation on differentiating loops for details and suggested alternatives.

In [18]:
gt = mi.TensorXf(np.ones((3,wavefront_size), dtype=np.float32))

In [19]:
print("training...")
params = mi.traverse(integrator)
opt = mi.ad.Adam(lr=LR, params={'sigmat': params['sigmat'], 'sh_coeffs': params['sh_coeffs']})
params.update(opt)

losses = []
intermediate_images = []

for stage in range(NUM_STAGES):
    print(f"Stage {stage+1:02d}, feature voxel grids resolution -> {opt['sigmat'].shape[0]}")

    for it in range(NUM_iterations_per_stage):
        total_loss = 0.0
        images = []

        img,_,_ = integrator.sample(mode=dr.ADMode.Primal,scene=scene, ray=rays, sampler=sampler, δL=None, state_in=None, active=active)
        loss = dr.mean(dr.abs(img - gt))
        dr.backward(loss)
        total_loss += loss[0]

        # Store images at the end of every stage
        if it == NUM_iterations_per_stage- 1:
            dr.eval(img)
            images.append(img)

        losses.append(total_loss)
        print("loss: ", total_loss)
        opt.step()

        if not integrator.use_relu:
            opt['sigmat'] = dr.maximum(opt['sigmat'], 0.0)

        params.update(opt)
        print(f"  --> iteration {it+1:02d}: error={total_loss:6f}", end='\r')

    intermediate_images.append(images)

    # Upsample the 3D textures at every stage
    if stage < NUM_STAGES - 1:
        new_res = 2 * opt['sigmat'].shape[0]
        new_shape = [new_res, new_res, new_res]
        opt['sigmat']   = dr.upsample(opt['sigmat'],   new_shape)
        opt['sh_coeffs'] = dr.upsample(opt['sh_coeffs'], new_shape)
        params.update(opt)

print('')
print('Done')

training...
Stage 01, feature voxel grids resolution -> 16
hit [False, False, False, False, False, .. 65526 skipped .., False, False, False, False, False]
active [True, True, True, True, True, .. 65526 skipped .., True, True, True, True, True]


Exception: loop_process_state(): one of the supplied loop state variables of type Float is attached to the AD graph (i.e., grad_enabled(..) is true). However, propagating derivatives through multiple iterations of a recorded loop is not supported (and never will be). Please see the documentation on differentiating loops for details and suggested alternatives.